In [1]:
print ("Hello")

Hello


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# Set up ChromeDriver service
service = Service()
driver = webdriver.Chrome(service=service)

try:
    # Open the website
    driver.get("https://www.nse.co.ke/dataservices/market-statistics/")

    # Use WebDriverWait to safely wait for the Equities tab to be clickable
    wait = WebDriverWait(driver, 10)
    equities_tab = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Equities Statistics']")))

    # Click the tab
    equities_tab.click()

    # Wait for the content to load after clicking
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "rowgroup")))

    # Open CSV file for writing
    with open("Shares.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Company", "ISIN Code", "Volume", "Last Traded Price", "Change"])

        # Find the main container for the data rows, which is the <rowgroup>
        row_group = driver.find_element(By.TAG_NAME, "rowgroup")

        # Find all the rows (<row>) within the rowgroup
        data_rows = row_group.find_elements(By.TAG_NAME, "row")

        # Loop through each row and scrape the data
        for row in data_rows:
            cells = row.find_elements(By.CLASS_NAME, "cell")

            if len(cells) >= 5:
                company = cells[0].text
                isin_code = cells[1].text
                volume = cells[2].text
                last_traded_price = cells[3].text
                change = cells[4].text

                writer.writerow([company, isin_code, volume, last_traded_price, change])

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Always close the browser
    driver.quit()

An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7a2affce5+79861]
	GetHandleVerifier [0x0x7ff7a2affd40+79952]
	(No symbol) [0x0x7ff7a287cada]
	(No symbol) [0x0x7ff7a28d4457]
	(No symbol) [0x0x7ff7a28d471c]
	(No symbol) [0x0x7ff7a2928217]
	(No symbol) [0x0x7ff7a28fcb1f]
	(No symbol) [0x0x7ff7a2924f8b]
	(No symbol) [0x0x7ff7a28fc8b3]
	(No symbol) [0x0x7ff7a28c5272]
	(No symbol) [0x0x7ff7a28c6043]
	GetHandleVerifier [0x0x7ff7a2dbba1d+2946349]
	GetHandleVerifier [0x0x7ff7a2db5c9a+2922410]
	GetHandleVerifier [0x0x7ff7a2dd5a27+3052855]
	GetHandleVerifier [0x0x7ff7a2b1aace+189918]
	GetHandleVerifier [0x0x7ff7a2b22a6f+222591]
	GetHandleVerifier [0x0x7ff7a2b08b04+116244]
	GetHandleVerifier [0x0x7ff7a2b08cb9+116681]
	GetHandleVerifier [0x0x7ff7a2aef098+11176]
	BaseThreadInitThunk [0x0x7ff87940e8d7+23]
	RtlUserThreadStart [0x0x7ff87a808d9c+44]

